A **flow** is the basis of all Prefect workflows. A flow is a Python function decorated with a @flow decorator.

A **task** is a Python function decorated with a @task decorator. Tasks represent distinct pieces of work executed within a flow.

All Prefect workflows are defined within the context of a flow. Every Prefect workflow must contain at least one flow function that serves as the entrypoint for execution of the flow.

Flows can include calls to tasks as well as to child flows, which we call "subflows" in this context. At a high level, this is just like writing any other Python application: you organize specific, repetitive work into tasks, and call those tasks from flows.

In [1]:
import requests
from prefect import flow, task
import time

In [2]:
@task
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

In [3]:
@flow
def api_flow(url):
    fact_json = call_api(url)
    return fact_json

In [4]:
print(api_flow("https://catfact.ninja/fact"))

22:31:21.886 | INFO    | prefect.engine - Created flow run 'fanatic-goose' for flow 'api-flow'

22:31:22.698 | INFO    | Flow run 'fanatic-goose' - Created task run 'call_api-ded10bed-0' for task 'call_api'

22:31:22.700 | INFO    | Flow run 'fanatic-goose' - Executing 'call_api-ded10bed-0' immediately...

200


22:31:23.987 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()

22:31:24.293 | INFO    | Flow run 'fanatic-goose' - Finished in state Completed()

{'fact': "A cat's smell is their strongest sense, and they rely on this leading sense to identify people and objects; a feline's sense of smell is 14x better than a human's.", 'length': 163}


Basic flow configuration includes the ability to provide a name, description, and version and other options for the flow via flow arguments.

You specify flow configuration as arguments on the @flow decorator.

By design, tasks follow a very similar model to flows: you can independently assign tasks their own name and description.

In [5]:
@task(name="My Example Task", 
      description="An example task for a tutorial.",
      retries=5,
      retry_delay_seconds=60)
def my_task():
    print("Task1")

@flow(name="My Example Flow", 
      description="An example flow for a tutorial.")
def my_flow():
    my_task()

**Task runners** are responsible for running Prefect tasks within a flow. Each flow has a task runner associated with it. Depending on the task runner you use, the tasks within your flow can run sequentially, concurrently, or in parallel. You can even configure task runners to use distributed execution infrastructure such as a Dask cluster.

By default, the result of a task is a Python object, and execution of the task blocks the execution of the next task in a flow. To make sure that the tasks within your flow can run concurrently or in parallel, add .submit() to your task run. This method will return a PrefectFuture instead of a Python object. A PrefectFuture is an object that provides access to a computation happening in a task runner. 

The default task runner is the ConcurrentTaskRunner, which will run submitted tasks concurrently. If you don't specify a task runner, Prefect uses the ConcurrentTaskRunner.

In [6]:
@task
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

22:31:25.015 | INFO    | prefect.engine - Created flow run 'elastic-bison' for flow 'my-flow'

22:31:25.796 | INFO    | Flow run 'elastic-bison' - Created task run 'print_values-0bb9a2c3-0' for task 'print_values'

22:31:25.800 | INFO    | Flow run 'elastic-bison' - Submitted task run 'print_values-0bb9a2c3-0' for execution.

22:31:25.940 | INFO    | Flow run 'elastic-bison' - Created task run 'print_values-0bb9a2c3-1' for task 'print_values'

22:31:25.944 | INFO    | Flow run 'elastic-bison' - Submitted task run 'print_values-0bb9a2c3-1' for execution.

22:31:31.567 | INFO    | Task run 'print_values-0bb9a2c3-1' - Finished in state Completed()

22:31:33.792 | INFO    | Task run 'print_values-0bb9a2c3-0' - Finished in state Completed()

22:31:33.991 | INFO    | Flow run 'elastic-bison' - Finished in state Completed('All states completed.')

Sometimes you may want to intentionally run **tasks sequentially**. The built-in Prefect SequentialTaskRunner lets you do this.

When using non-default task runner, you must import the task runner into your flow script.

In [7]:
from prefect.task_runners import SequentialTaskRunner

@task(name="Sequential Task")
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(name="Sequential Flow",
      task_runner=SequentialTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

22:31:34.509 | INFO    | prefect.engine - Created flow run 'cinnamon-bettong' for flow 'Sequential Flow'

22:31:35.450 | INFO    | Flow run 'cinnamon-bettong' - Created task run 'Sequential Task-0bb9a2c3-0' for task 'Sequential Task'

22:31:35.453 | INFO    | Flow run 'cinnamon-bettong' - Executing 'Sequential Task-0bb9a2c3-0' immediately...

22:31:43.489 | INFO    | Task run 'Sequential Task-0bb9a2c3-0' - Finished in state Completed()

22:31:43.641 | INFO    | Flow run 'cinnamon-bettong' - Created task run 'Sequential Task-0bb9a2c3-1' for task 'Sequential Task'

22:31:43.646 | INFO    | Flow run 'cinnamon-bettong' - Executing 'Sequential Task-0bb9a2c3-1' immediately...

22:31:49.009 | INFO    | Task run 'Sequential Task-0bb9a2c3-1' - Finished in state Completed()

22:31:49.156 | INFO    | Flow run 'cinnamon-bettong' - Finished in state Completed('All states completed.')

You can also run tasks using **parallel or distributed** execution by using the Dask or Ray task runners available through **Prefect Collections**.

For example, you can achieve parallel task execution, even on in a local execution environment, but using the DaskTaskRunner.

1. Install the prefect-dask collection with pip install prefect-dask.
2. Switch your task runner to the DaskTaskRunner.
3. Call .submit on the task instead of calling the task directly. This submits the task to the task runner rather than running the task in-process.


In [8]:
!pip install prefect-dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 754.6 kB/s eta 0:00:0031m18.5 MB/s eta 0:00:01
  Using cached msgpack-1.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (322 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:01:01


In [10]:
from prefect_dask.task_runners import DaskTaskRunner

@task(name="Dask Task")
def print_values(values):
    for value in values:
        time.sleep(0.5)
        print(value, end="\r")

@flow(name="Dask Flow",
      task_runner=DaskTaskRunner())
def my_flow():
    print_values.submit(["AAAA"] * 15)
    print_values.submit(["BBBB"] * 10)

if __name__ == "__main__":
    my_flow()

/root/.local/share/virtualenvs/overview-xFZwEpOD/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'Dask Task' and defined at '/tmp/ipykernel_101543/4008450591.py:3' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/root/.local/share/virtualenvs/overview-xFZwEpOD/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'Dask Flow' and defined at '/tmp/ipykernel_101543/4008450591.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


22:43:21.537 | INFO    | prefect.engine - Created flow run 'resilient-kingfisher' for flow 'Dask Flow'

22:43:21.544 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`

22:43:21.558 | INFO    | distributed.scheduler - State start

22:43:21.573 | INFO    | distributed.scheduler -   Scheduler at:     tcp://127.0.0.1:35471

...

22:43:35.119 | INFO    | distributed.core - Removing comms to tcp://127.0.0.1:32999

22:43:35.127 | INFO    | distributed.scheduler - Lost all workers

22:43:35.820 | INFO    | distributed.scheduler - Scheduler closing...

22:43:35.822 | INFO    | distributed.scheduler - Scheduler closing all comms

22:43:35.824 | INFO    | Flow run 'resilient-kingfisher' - Finished in state Completed('All states completed.')

Because the DaskTaskRunner uses multiprocessing, it must be protected by an if __name__ == "__main__": guard when used in a script.

When using the DaskTaskRunner, Prefect is submitting each task run to a Dask cluster object. The Dask scheduler then determines when and how each individual run should be executed (with the constraint that the order matches the execution graph that Prefect provided).

This means the only way to force Dask to walk the task graph in a particular order is to configure Prefect dependencies between your tasks.